#### 패키지 로딩하기

In [1]:
from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix
import pandas as pd
import numpy as np
import os

#### 데이터 준비 및 전처리

In [2]:
rating_file_path  = os.getenv("HOME") + "/aiffel/recommendata_iu/data/ml-1m/ratings.dat" # file 경로 지정
ratings_cols      = ["user_id", "movie_id", "rating", "timestamp"]                       # columns naming
ratings           = pd.read_csv(filepath_or_buffer = rating_file_path,                   # 파일 불러오기
                                sep                = "::", 
                                names              = ratings_cols, 
                                engine             = "python")
orginal_data_size = len(ratings)                                                         # orginal_data_size 보기
ratings.head()                                                                           # ratings 위의 5개 데이터 보기

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
ratings            = ratings[ratings["rating"] >= 3]                                        # rating이 3점 이상인 것만 추려낸다.
filtered_data_size = len(ratings)                                                           # filtered_data_size 보기

print(f"orginal_data_size : {orginal_data_size}, filtered_data_size :{filtered_data_size}") # 2개의 data_size를 각각 보여주기
print(f"Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}")           # 3점 이상의 데이터 비율 보기

orginal_data_size : 1000209, filtered_data_size :836478
Ratio of Remaining Data is 83.63%


In [4]:
# rating의 컬럼 이름을 count로 바꾼다.
ratings.rename(columns = {"rating" : "count"}, inplace = True) # 칼럼 이름을 바꿔서 원본에 덮어쓰기
ratings.head()

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [5]:
# movie 정보를 가져옵니다.
movie_file_path = os.getenv("HOME") + "/aiffel/recommendata_iu/data/ml-1m/movies.dat" # file 경로 지정
cols            = ["movie_id", "title", "genre"]                                      # columns naming
movies          = pd.read_csv(filepath_or_buffer = movie_file_path,                   # file 불러오기
                              sep                = "::", 
                              names              = cols, 
                              engine             = "python",
                              encoding           = "ISO-8859-1")
movies["title"] = movies["title"].str.lower()                                         # title의 데이터를 전부 소문자로 바꾸기
movies["genre"] = movies["genre"].str.lower()                                         # genre의 데이터를 전부 소문자로 바꾸기
movies.head()

,movie_id,title,genre
0,1,toy story (1995),animation|children's|comedy
1,2,jumanji (1995),adventure|children's|fantasy
2,3,grumpier old men (1995),comedy|romance
3,4,waiting to exhale (1995),comedy|drama
4,5,father of the bride part ii (1995),comedy


In [6]:
# title과 genre의 정보도 얻기위해 ratings을 기준으로 movies를 합친다. 
movies_data = pd.merge(ratings, movies, on = "movie_id", how = "left")
movies_data

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,one flew over the cuckoo's nest (1975),drama
1,1,661,3,978302109,james and the giant peach (1996),animation|children's|musical
2,1,914,3,978301968,my fair lady (1964),musical|romance
3,1,3408,4,978300275,erin brockovich (2000),drama
4,1,2355,5,978824291,"bug's life, a (1998)",animation|children's|comedy
...,...,...,...,...,...,...
836473,6040,1090,3,956715518,platoon (1986),drama|war
836474,6040,1094,5,956704887,"crying game, the (1992)",drama|romance|war
836475,6040,562,5,956704746,welcome to the dollhouse (1995),comedy|drama
836476,6040,1096,4,956715648,sophie's choice (1982),drama


#### 데이터 탐색

In [7]:
# movie_data의 각 칼럼 갯수 보기
print("movie_data 의 user_id 갯수 :",   movies_data["user_id"].nunique())
print("movie_data 의 movie_id 갯수 :",  movies_data["movie_id"].nunique())
print("movie_data 의 count 갯수 :",     movies_data["count"].nunique()) # 3, 4, 5 만 존재
print("movie_data 의 timestamp 갯수 :", movies_data["timestamp"].nunique())
print("movie_data 의 title 갯수 :",     movies_data["title"].nunique())
print("movie_data 의 genre 갯수 :",     movies_data["genre"].nunique())

movie_data 의 user_id 갯수 : 6039
movie_data 의 movie_id 갯수 : 3628
movie_data 의 count 갯수 : 3
movie_data 의 timestamp 갯수 : 412911
movie_data 의 title 갯수 : 3628
movie_data 의 genre 갯수 : 301


In [8]:
# 가장 인기있는 movies 상위 30개 확인하기
movies_count = movies_data.groupby("movie_id")["user_id"].count() # movies_id로 grouping 후 user_id 세기
movies_count.sort_values(ascending = False).head(30)              # 내림차순하여 상위 30개 보기

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

#### 모델 검증을 위한 사용자 초기 정보 세팅

In [9]:
# 내가 좋아하는 영화 5개 선정하여 데이터셋 만들기 (comedy 5개 골라서 넣기)
my_favorite = ["Warcraft (2015)", 
               "The Hunger Games: Mockingjay - Part 2 (2015)", 
               "Aladdin (2019)", 
               "The Chronicles of Narnia: The Lion, the Witch and the Wardrobe (2005)", 
               "Harry Potter And The Deathly Hallows: Part 2 (2011)"]

# "my"라는 user_id를 가진 사람이 위 5개 영화를 전부 5번씩 봤다고 가정하기
my_list = pd.DataFrame({
    "user_id"  : ["my"] * 5, 
    "movie_id" : [3953., 3954., 3955., 3956., 3957.], 
    "count"    : [5] * 5, 
    "title"    : my_favorite, 
    "genre"    : ["fantasy|action", "fantasy|action", "fantasy|romantic|adventure", "fantasy|adventure|family", "fantasy|adventure"]
})

# user_id에 "my"가 없다면 my_list를 추가한다.
if not movies_data.isin({"user_id" : ["my"]})["user_id"].any():
    movies_data = movies_data.append(my_list)

movies_data.tail(10) # my_list가 잘 붙었는지 확인하기

,user_id,movie_id,count,timestamp,title,genre
836473,6040,1090.0,3,956715518.0,platoon (1986),drama|war
836474,6040,1094.0,5,956704887.0,"crying game, the (1992)",drama|romance|war
836475,6040,562.0,5,956704746.0,welcome to the dollhouse (1995),comedy|drama
836476,6040,1096.0,4,956715648.0,sophie's choice (1982),drama
836477,6040,1097.0,4,956715569.0,e.t. the extra-terrestrial (1982),children's|drama|fantasy|sci-fi
0,my,3953.0,5,NaN,Warcraft (2015),fantasy|action
1,my,3954.0,5,NaN,The Hunger Games: Mockingjay - Part 2 (2015),fantasy|action
2,my,3955.0,5,NaN,Aladdin (2019),fantasy|romantic|adventure
3,my,3956.0,5,NaN,"The Chronicles of Narnia: The Lion, the Witch ...",fantasy|adventure|family
4,my,3957.0,5,NaN,Harry Potter And The Deathly Hallows: Part 2 (...,fantasy|adventure


#### 모델에 활용하기 위한 전처리

In [10]:
# 고유한 유저, 영화를 찾아내기
user_unique  = movies_data["user_id"].unique()
movie_unique = movies_data["title"].unique()

# 유저, 영화를 indexing 하기, idx는 index의 약자이다.
user_to_idx  = {v:k for k, v in enumerate(user_unique)}
movie_to_idx = {v:k for k, v in enumerate(movie_unique)}

# indexing 확인하기
print(user_to_idx["my"])
print(movie_to_idx["Warcraft (2015)"])

6039
3628


In [11]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구한다.
# 혹시 정상적으로 인덱싱되지 않은 행이 있다면 인덱스가 NaN이 되니 dropna()로 제거한다.
temp_user_data = movies_data["user_id"].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(movies_data):
    print("user_id column indexing OK!")
    movies_data["user_id"] = temp_user_data
else:
    print("user_id column indexing Fail!")

# movie_to_idx을 통해 movie_id 컬럼도 동일한 방식으로 인덱싱을 해준다.
temp_movie_data = movies_data["title"].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(movies_data):
    print("movies_id column indexing OK!")
    movies_data["title"] = temp_movie_data
else:
    print("movies_id column indexing Fail!")
    
ratings

user_id column indexing OK!
movies_id column indexing OK!


,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


#### CSR matrix를 직접 만들어 봅시다.

In [12]:
num_user  = movies_data["user_id"].nunique()
num_movie = movies_data["title"].nunique()

csr_data  = csr_matrix((movies_data["count"], (movies_data.user_id, movies_data.title)), shape = (num_user, num_movie))
csr_data

<6040x3633 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [13]:
# implicit 라이브러리에서 권장하고 있는 부분이다.
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["KMP_DUPLICATE_LIB_OK"] = "True"
os.environ["MKL_NUM_THREADS"]      = "1"

In [14]:
# # Implicit AlternatingLeastSquares 모델 선언
# factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지 정한다. default = 100
# regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지 정한다. default = 0.01
# use_gpu : GPU를 사용할 것인지 정한다. default = False
# iterations : epochs와 같은 의미로 데이터를 몇 번 반복해서 학습할 것인지 정한다. default = 15
# factors, iterations를 늘릴수록 학습데이터를 잘 학습하게 되지만 과적합의 우려가 있으므로 적절한 값을 찾아야 한다.
als_model = AlternatingLeastSquares(factors = 5000, regularization = 0.01, use_gpu = False, iterations = 20, dtype = np.float32)

In [15]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해준다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3633x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [16]:
# model 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/20 [00:00<?, ?it/s]

In [17]:
# my와 sabrina의 벡터를 어떻게 만들었는지 확인하기
my, warcraft               = user_to_idx["my"], movie_to_idx["Warcraft (2015)"]           # my와 warcraft의 value를 꺼내온다.
my_vector, warcraft_vector = als_model.user_factors[my], als_model.item_factors[warcraft] # 훈련된 모델에 my와 warcraft에 해당하는 벡터를 꺼내온다.

In [18]:
# my의 벡터
my_vector

array([ 0.03005343, -0.21316122, -0.23139833, ..., -0.31946602,
       -0.13583732,  0.07448628], dtype=float32)

In [19]:
# warcraft의 벡터
warcraft_vector

array([ 0.00607311,  0.00389295,  0.00248817, ..., -0.00042185,
        0.00718522,  0.00568625], dtype=float32)

In [20]:
# my와 warcraft의 벡터를 곱하면 무슨값이 나오는지 확인하기
np.dot(my_vector, warcraft_vector)

0.93805957

In [21]:
# 다른 영화를 내가 좋아할까?
other        = movie_to_idx["Aladdin (2019)"]
other_vector = als_model.item_factors[other]
np.dot(my_vector, other_vector)

0.9344188

#### 내가 좋아하는 영화와 비슷한 영화를 추천받기

In [22]:
# movie_id와 movie_to_idx를 이용하여 영화 이름을 얻는다.
idx_to_movie = {v:k for k, v in movie_to_idx.items()} # als_model.similar_items에서 나오는 값을 이용하기 위해 idx_to_movie를 만들어준다.

# 영화를 넣으면 유사한 영화를 추천하는 함수를 만든다.
def get_similar_movie(movie_name : str):
    movie_id      = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id) # 나오는 값은 (movie_id, 유사도)의 형태인 tuple이 반환된다.
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie] # 나온 값(유사도가 높은 movie_id순으로 된)을 movie 이름으로 바꿔준다.
    return similar_movie

In [23]:
# 영화를 추천받아보자
get_similar_movie("Warcraft (2015)")

['Warcraft (2015)',
 'little city (1998)',
 'Harry Potter And The Deathly Hallows: Part 2 (2011)',
 'second best (1994)',
 'Aladdin (2019)',
 'bluebeard (1944)',
 'The Chronicles of Narnia: The Lion, the Witch and the Wardrobe (2005)',
 'beauty (1998)',
 'joyriders, the (1999)',
 'The Hunger Games: Mockingjay - Part 2 (2015)']

#### 유저에게 영화 추천하기

In [24]:
user = user_to_idx["my"]

# recommend에서는 user * item CSR Matrix를 받습니다.

# 나오는 값은 (movie_id, 유사도)의 형태인 tuple이 반환된다.
movie_recommend = als_model.recommend(user, csr_data, N = 15, filter_already_liked_items = True) 

# 나온 값(유사도가 높은 movie_id순으로 된)을 movie 이름으로 바꿔준다. 
[idx_to_movie[i[0]] for i in movie_recommend]

['murder! (1930)',
 'ulysses (ulisse) (1954)',
 'rent-a-kid (1995)',
 'condition red (1995)',
 'dangerous ground (1997)',
 'fever pitch (1997)',
 "white man's burden (1995)",
 'adrenalin: fear the rush (1996)',
 'chill factor (1999)',
 'go now (1995)',
 'tough guys (1986)',
 'larger than life (1996)',
 'machine, the (1994)',
 'schlafes bruder (brother of sleep) (1995)',
 'man from down under, the (1943)']

In [27]:
# 추천해준 값 중 첫번째 것을 왜 추천해줬는지 궁금하다.
# 기록을 남긴 데이터 중 이 추천에 기여한 정도를 확인한다.
first_recommend = movie_to_idx["murder! (1930)"]
explain         = als_model.explain(user, csr_data, itemid = first_recommend)

# 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도(합이 콘텐츠의 점수가 된다)를 반환한다.
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('The Chronicles of Narnia: The Lion, the Witch and the Wardrobe (2005)',
  0.011061408161239947),
 ('Harry Potter And The Deathly Hallows: Part 2 (2011)', 0.008213900501744384),
 ('Aladdin (2019)', 0.005436494314643714),
 ('The Hunger Games: Mockingjay - Part 2 (2015)', 0.004610221687627557),
 ('Warcraft (2015)', 0.002251436114249338)]

#### 회고록

In [26]:
# 실습 데이터는 2개가 있었고, 그 중 rating 문서를 기반으로 실습이 진행된 것 같음.
# 문제는 영화 제목은 movie 문서에 있었는데 rating만 가지고 실습을 하다보니 영화 제목이 글자가 아닌 숫자로 나옴.
# 어떻게 이 문제를 해결할까 고민하던 중 두 문서를 merge하는 방법을 떠올려서 잘 합쳤으나, 실습결과 똑같이 숫자가 나옴.
# 멍청하게도 바꾼 문서에서 title을 가지고 진행했어야하나, 아직도 movie_id를 가지고 코드를 구현해서 제목이 안 나왔었음.
# 코드를 바꾸고, 합친 문서를 가지고 실습한 결과 내가 원하는대로 추천알고리즘의 결과가 영화 제목으로 잘 나옴.
# 또한, 노드처럼 말고 내가 좋아하는 영화에 대한 정보를 DataFrame에 맞게 넣고 싶어서 9번 셀에 영화 정보를 자세하게 추가함.
# 그리고 모델 학습에서 factors의 값을 4자리 수까지 올려서 학습을 했더니 내가 본 영화와 나의 행렬 내적 값이 0.9점대 이상으로 올라감.
# 내가 추가한 다른 영화와 나와 어떠한지 값을 비교해보니 0.9점대로 높게 나왔다.
# 그리고 내가 본 영화와 비슷한 것을 추천 받아보니 비슷한 것들이 나온듯 하다.
# 추천해준 영화 중 첫번째 영화에 기여도가 높은 영화들을 보니 내가 추가한 것 5개가 나왔다.